In [26]:
import numpy as np
import time

In [7]:
import json
def load_taillard():
    path = 'datasets/taillard.txt'
    with open(path) as f:
        n = int(f.readline())
        tests = []
        tests_json = []
        for _ in range(n):
            test = {}
            next(f)
            nb_jobs, nb_machines, seed, ub, lb = [int(i) for i in f.readline().strip().split(" ") if i != ""]
            test["nb_jobs"] = nb_jobs
            test["nb_machines"] = nb_machines
            test["seed"] = seed
            test["upper_bound"] = ub
            test["lower_bound"] = lb
            next(f)
            jobs = []
            for _ in range(nb_machines):
                process_times = [int(i) for i in f.readline().strip().split(" ") if i != ""]
                jobs.append(process_times)
            test["jobs"] = jobs
            tests_json.append(test)
            tests.append((nb_jobs, nb_machines, seed, ub, lb, list(zip(*jobs))))
    with open('datasets/taillard.json', 'w') as f:
        json.dump(tests_json, f)
    return tests
_ = load_taillard()

In [28]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [29]:
# Define the Node structure of the seach tree that we will be using
class Node:
    def __init__(self, jobs, remaining_jobs, parent=None, lower_bound=1e100):
        self.jobs = jobs
        self.remaining_jobs = remaining_jobs
        self.parent = parent
        self.lower_bound = lower_bound
    def __str__(self):
        return f"Node(jobs={self.jobs}, remaining_jobs={self.remaining_jobs}, lower_bound={self.lower_bound})"

In [64]:
def branch_and_bound(processing_times, initial_solution, initial_cost, lower_bound=0, upper_bound=None):
    jobs, machines = processing_times.shape
    # Initialize the nodes list to the `root_node`
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            child_lower_bound = evaluate_sequence(child_jobs, processing_times)
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound
                    continue
            # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
            if child_lower_bound < best_solution_cost:
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
            
        i += 1
        if i % 10000 == 0:
            print(i, best_solution, best_solution_cost, 'Stack length', len(nodes))
    return best_solution, best_solution_cost, i

In [65]:
def branch_and_bound_pure(processing_times,initial_solution, initial_cost):
    jobs, machines = processing_times.shape
    # Initialize the nodes list to the `root_node`
    root_node = Node([], set(range(jobs)))
    best_solution = initial_solution.copy()
    best_solution_cost = initial_cost
    nodes = [root_node]
    i = 1
    while nodes:
        node = nodes.pop()
        # Explore neighbours of the node `node`
        for job in node.remaining_jobs:
            child_jobs = node.jobs + [job]
            child_remaining_jobs = node.remaining_jobs - {job}
            # If the child node is a leaf node (i.e., all jobs have been assigned) then calculate its cost
            if not child_remaining_jobs:
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                if child_lower_bound < best_solution_cost:
                    best_solution = child_jobs
                    best_solution_cost = child_lower_bound   
                    continue
            else:
                # If the child node is not a leaf then calculate its lower bound and compare it with current `best_solution_cost`
                child_lower_bound = evaluate_sequence(child_jobs, processing_times)
                child_node = Node(child_jobs, child_remaining_jobs, parent=node, lower_bound=child_lower_bound)
                nodes.append(child_node)
        i += 1
        if i % 1000 == 0:
            print(i, best_solution, len(nodes))
    return best_solution, best_solution_cost, i

In [51]:
data = load_taillard()
nb_jobs, nb_machines, seed, ub, lb, processing_times = data[0]
instance = np.array(processing_times)
initial_solution = np.array(range(nb_jobs))

initial_cost = evaluate_sequence(initial_solution, instance)


In [ ]:
start_time = time.time()
best_solution, best_cost, i = branch_and_bound_pure(instance, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound pure:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')

In [ ]:
start_time = time.time()
best_solution, best_cost, i = branch_and_bound(instance, initial_solution, initial_cost)
elapsed_time = time.time() - start_time

print(f'Results of Branch & Bound:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'No. Nodes visited is {i}.')
print(f'Elapsed time of {elapsed_time} seconds.')